In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import subprocess

import tenseal as ts


In [133]:
n_parts = 16
context = [256, 512, 1024][0]
m_entries = 8
tag = "123456"

key = {None:"N/A",  256:"keys/def_80_256_1", 512:"keys/def_80_512_1", 1024:"keys/def_80_2014_1"}[context]
prec = 6
padd = int(np.log2(n_parts))
lower = -1.0
upper = 1.0

vec = np.random.uniform(-1., 1., size = m_entries)
txt = " ".join(["%0.6f"%a for a in vec])

encfile = f"temp/_tfhe_{tag}_{context}_{n_parts}_{m_entries}.enc"

command = f"target/release/enc_vector {key} {encfile} {prec} {padd} {lower} {upper} {m_entries} {txt}"
print(command)

out = subprocess.getoutput(command)
print(out)


target/release/enc_vector keys/def_80_256_1 temp/_tfhe_123456_256_16_8.enc 6 4 -1.0 1.0 8 0.057708 0.629166 -0.352151 -0.849674 0.495149 -0.710024 0.546311 0.537201
# sk_path: keys/def_80_256_1
# prec: 6
# padd: 4
# lower: -1
# upper: 1
# m_entries: 8
# create an encoder... 

x: 0.057708
x: 0.629166
x: -0.352151
x: -0.849674
x: 0.495149
x: -0.710024
x: 0.546311
x: 0.537201
v: [0.057708, 0.629166, -0.352151, -0.849674, 0.495149, -0.710024, 0.546311, 0.537201]
Loss of precision during encrypt: 3 bit(s) with 6 bit(s) of message originally. Consider increasing the dimension the reduce the amount of noise needed.
keys/def_80_256_1 8 433 793


In [134]:
!ls -l temp/_tfhe*

-rw-r--r-- 1 jovyan users 42921 Dec 25 11:37 temp/_tfhe_123456_256_16_8.enc


In [132]:
for tag in range(25):
    for n_parts in [2,4,8,16]:
        for context in [256, 512]:
            for m_entries in [1, 10, 100]:

                key = {None:"N/A",  256:"keys/def_80_256_1", 512:"keys/def_80_512_1", 1024:"keys/def_80_2014_1"}[context]
                prec = 6
                padd = int(np.log2(n_parts))
                lower = -1.0
                upper = 1.0

                vec = np.random.uniform(-1., 1., size = m_entries)
                txt = " ".join(["%0.6f"%a for a in vec])

                #encfile = f"temp/test{tag}_{context}_{n_parts}_{m_entries}.enc"
                encfile = f"temp/tfhe_{tag}_{context}_{n_parts}_{m_entries}.enc"

                command = f"target/release/enc_vector {key} {encfile} {prec} {padd} {lower} {upper} {m_entries} {txt}"
                #print(command)

                out = subprocess.getoutput(command)
                #print(out)


In [135]:
n_parts = 16
poly_mod_degree = [4096,8192][0]
m_entries = 8
tag = 123456

if poly_mod_degree==4096:
    coeff_mod_bit_sizes = [40, 20, 40]
    ckks_context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
    ckks_context.global_scale = 2 ** 20
else:
    #coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
    coeff_mod_bit_sizes = [60,40,40,60]
    ckks_context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
    #context.global_scale = 2 ** 21
    ckks_context.global_scale = 2 ** 40

ckks_context.generate_galois_keys()

vec = np.random.uniform(-1., 1., size = m_entries)
filename = f"{tag}_{poly_mod_degree}_{n_parts}_{m_entries}"

encfile = f"temp/_ckks_"+filename+".enc"
txtfile = f"temp/_text_"+filename+".txt"

print(encfile)
print(txtfile)

vec_ckks = ts.ckks_vector(ckks_context, vec)

with open(encfile, "wb") as f:
    f.write(vec_ckks.serialize())

np.savetxt(txtfile, vec) 


temp/_ckks_123456_4096_16_8.enc
temp/_text_123456_4096_16_8.txt


In [136]:
!ls -l temp/_ckks*

-rw-r--r-- 1 jovyan users 80914 Dec 25 11:37 temp/_ckks_123456_4096_16_8.enc


In [139]:
for tag in range(25):
    for n_parts in [2,4,8,16]:
        for poly_mod_degree in [4096,8192]:
            for m_entries in [1, 10, 100]:

                if poly_mod_degree==4096:
                    coeff_mod_bit_sizes = [40, 20, 40]
                    ckks_context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
                    ckks_context.global_scale = 2 ** 20
                else:
                    #coeff_mod_bit_sizes = [40, 21, 21, 21, 21, 21, 21, 40]
                    coeff_mod_bit_sizes = [60,40,40,60]
                    ckks_context = ts.context(ts.SCHEME_TYPE.CKKS, poly_mod_degree, -1, coeff_mod_bit_sizes)
                    #context.global_scale = 2 ** 21
                    ckks_context.global_scale = 2 ** 40

                ckks_context.generate_galois_keys()

                vec = np.random.uniform(-1., 1., size = m_entries)
                filename = f"{tag}_{poly_mod_degree}_{n_parts}_{m_entries}"
                encfile = f"temp/ckks_"+filename+".enc"
                txtfile = f"temp/text_"+filename+".txt"
                
                #print(encfile)
                #print(txtfile)

                vec_ckks = ts.ckks_vector(ckks_context, vec)

                with open(encfile, "wb") as f:
                    f.write(vec_ckks.serialize())

                np.savetxt(txtfile, vec) 
          

In [158]:
!ls temp/tfhe0*

ls: cannot access 'temp/tfhe0*': No such file or directory


In [149]:
!ls -l temp/*.* | grep 'temp/_'


-rw-r--r-- 1 jovyan users   80914 Dec 25 11:37 temp/_ckks_123456_4096_16_8.enc
-rw-r--r-- 1 jovyan users     206 Dec 25 11:37 temp/_text_123456_4096_16_8.txt
-rw-r--r-- 1 jovyan users   42921 Dec 25 11:37 temp/_tfhe_123456_256_16_8.enc


[['ckks', '0', '4096', '16', '100'],
 ['ckks', '0', '4096', '16', '10'],
 ['ckks', '0', '4096', '16', '1'],
 ['ckks', '0', '4096', '2', '100'],
 ['ckks', '0', '4096', '2', '10'],
 ['ckks', '0', '4096', '2', '1'],
 ['ckks', '0', '4096', '4', '100'],
 ['ckks', '0', '4096', '4', '10'],
 ['ckks', '0', '4096', '4', '1'],
 ['ckks', '0', '4096', '8', '100']]

In [178]:
file_list = subprocess.getoutput("ls -l temp/*.* | grep -v 'temp/_'")
#print(tfhe_list)

df = pd.DataFrame([row.split(" ")[-1].split("/")[-1].split(".")[0].split("_")[:] for row in file_list.split("\n")], 
                 columns=["scheme","tag","poly","parts","vec len"])
df["file size"] = [int(row.split(" ")[-5]) for row in file_list.split("\n")]

df.head()


,scheme,tag,poly,parts,vec len,file size
0,ckks,0,4096,16,100,80894
1,ckks,0,4096,16,10,80886
2,ckks,0,4096,16,1,80901
3,ckks,0,4096,2,100,80944
4,ckks,0,4096,2,10,80909


In [185]:
idxs = df.scheme=="text"
df.loc[idxs,"scheme"] = "plain"
df.loc[idxs,"poly"] = "text"
df["context"] = df["scheme"]+" "+df["poly"].apply(str)
df.head()


,scheme,tag,poly,parts,vec len,file size,context
0,ckks,0,4096,16,100,80894,ckks 4096
1,ckks,0,4096,16,10,80886,ckks 4096
2,ckks,0,4096,16,1,80901,ckks 4096
3,ckks,0,4096,2,100,80944,ckks 4096
4,ckks,0,4096,2,10,80909,ckks 4096


In [186]:
df.groupby(["context", "parts", "vec len"])["file size"].mean().reset_index().pivot(index=["context","parts"], columns="vec len", values="file size")


vec len                   1         10         100
context    parts                                  
ckks 4096  16      80899.36   80900.56    80900.80
           2       80893.16   80912.48    80917.28
           4       80904.00   80902.84    80897.64
           8       80892.04   80917.12    80901.72
ckks 8192  16     334331.52  334330.88   334322.64
           2      334345.32  334321.72   334266.72
           4      334257.08  334224.52   334349.76
           8      334348.04  334313.36   334318.44
plain text 16         25.48     255.10     2549.94
           2          25.44     254.84     2549.60
           4          25.48     254.88     2548.60
           8          25.54     255.24     2549.58
tfhe 256   16       5459.44   53628.20   535311.48
           2        5459.84   53623.24   535304.16
           4        5456.76   53635.72   535349.96
           8        5454.88   53626.96   535325.48
tfhe 512   16      10685.92  105880.60  1057878.48
           2       10684.44  105877.64  1057886.72
           4       10680.28  105884.36  1057849.84
           8       10677.88  105878.28  1057951.24

In [210]:
tmp = df.groupby(["context","vec len"])["file size"].mean().reset_index().pivot(index=["context"], columns="vec len", values="file size")
tmp["3591"] = tmp["100"] * 3591/100
tmp.loc[tmp.index=="ckks 4096","3591"] *= 2*100/3591
tmp.loc[tmp.index=="ckks 8192","3591"] *= 100/3591
tmp = tmp.iloc[[3,4,0,1,2]].round()/1000
tmp.to_csv("tbls/filesize.csv")
tmp


vec len,1,10,100,3591
context,,,,
tfhe 256,5.458,53.629,535.323,19223.441
tfhe 512,10.682,105.880,1057.892,37988.886
ckks 4096,80.897,80.908,80.904,161.809
ckks 8192,334.320,334.298,334.314,334.314
plain text,0.025,0.255,2.549,91.550
